In [ ]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os 
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory 
from langchain.chains import ConversationalRetrievalChain

In [2]:
%pwd

'c:\\Users\\hp\\Desktop\\PProject\\End-to-End-Source-code-Analysis-GenAi\\research'

In [3]:
!mkdir test_repo

A subdirectory or file test_repo already exists.


In [4]:
repo_path = "test_repo/"

repo = Repo.clone_from("https://github.com/rohitmukati/End-to-End-Medical-Chatbot-Generative-AI", to_path=repo_path)

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/rohitmukati/End-to-End-Medical-Chatbot-Generative-AI test_repo/
  stderr: 'fatal: destination path 'test_repo' already exists and is not an empty directory.
'

In [5]:
repo_path = "test_repo/"

In [6]:
loader = GenericLoader.from_filesystem(repo_path,
                                       glob = "**/*",
                                       suffixes = [".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
                                       
                                       
                                       )

In [7]:
documents = loader.load()

In [8]:
documents

[Document(page_content='from flask import Flask, render_template, jsonify, request\nfrom src.helper import download_huggingface_embedding\nfrom langchain_openai import OpenAI\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nimport os\n\nfrom dotenv import load_dotenv\nload_dotenv()\n\n\n\napp = Flask(__name__)\n\nPINECONE_API_KEY = os.environ.get(\'PINECONE_API_KEY\')\nOPENAI_API_KEY = os.environ.get(\'OPENAI_API_KEY\')\n\nos.environ[\'PINECONE_API_KEY\'] = PINECONE_API_KEY\nos.environ[\'OPENAI_API_KEY\'] = OPENAI_API_KEY\n\nembedding = download_huggingface_embedding()\n \n## loading existing index from pinecone database \n\nindex_name = "medicalbot"\ndocsearch = PineconeVectorStore.from_existing_index(\n    index_name = index_name,\n    embed

In [9]:
len(documents)

7

In [10]:
documents[0]

Document(page_content='from flask import Flask, render_template, jsonify, request\nfrom src.helper import download_huggingface_embedding\nfrom langchain_openai import OpenAI\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nimport os\n\nfrom dotenv import load_dotenv\nload_dotenv()\n\n\n\napp = Flask(__name__)\n\nPINECONE_API_KEY = os.environ.get(\'PINECONE_API_KEY\')\nOPENAI_API_KEY = os.environ.get(\'OPENAI_API_KEY\')\n\nos.environ[\'PINECONE_API_KEY\'] = PINECONE_API_KEY\nos.environ[\'OPENAI_API_KEY\'] = OPENAI_API_KEY\n\nembedding = download_huggingface_embedding()\n \n## loading existing index from pinecone database \n\nindex_name = "medicalbot"\ndocsearch = PineconeVectorStore.from_existing_index(\n    index_name = index_name,\n    embedd

In [12]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,
    chunk_size=500,
    chunk_overlap=200
)

In [13]:
texts = documents_splitter.split_documents(documents)

In [14]:
texts

[Document(page_content='from flask import Flask, render_template, jsonify, request\nfrom src.helper import download_huggingface_embedding\nfrom langchain_openai import OpenAI\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nimport os\n\nfrom dotenv import load_dotenv\nload_dotenv()', metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content="from dotenv import load_dotenv\nload_dotenv()\n\n\n\napp = Flask(__name__)\n\nPINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')\nOPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')\n\nos.environ['PINECONE_API_KEY'] = PINECONE_API_KEY\nos.environ['OPENAI_API_KEY'] = OPENAI_API_KEY\n\nembedding = download_huggingface_embedding()\n \n## loading existing index

In [15]:
len(texts)

18

In [16]:
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [17]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [18]:
embedding = OpenAIEmbeddings(disallowed_special=())

In [19]:
vectors = Chroma.from_documents(texts, embedding=embedding, persist_directory='.DB')

In [20]:
vectors.persist()

In [21]:
## llm
llm = ChatOpenAI()

In [22]:
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

In [23]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectors.as_retriever(search_type="mmr", search_kwargs={"k":8}), memory=memory)

In [26]:
question = "Can you write me function of adding two numbers in python?"

In [27]:
result = qa(question)
print(result["answer"])

I don't have that information.
